In [ ]:
from glob import glob
import astropy.io.fits as fits
import shutil
import os
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table, unique, vstack
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
import multiprocessing as mp
from drizzlepac import astrodrizzle, tweakreg
from stwcs import updatewcs
from acstools import calacs

os.environ['PATH'] = '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin:/home/avinash/part1/softwares/dolphot2.0/bin'
os.environ['PATH'] += ":/home/avinash/part1/venv/hst/bin/:/home/avinash/part4/HST/pkgs/hstcal/bin/"

os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = '/home/avinash/part4/HST/CRDS/'
os.environ['iref'] = '/home/avinash/part4/HST/CRDS/references/hst/wfc3/'
os.environ['jref'] = '/home/avinash/part4/HST/CRDS/references/hst/acs/'

from pydol.photometry.scripts import cmdtools

In [ ]:
obs_table = Observations.query_criteria(proposal_id=["10776"],objectname="M82",filters= ['F435W', 'F555W', 'F814W'])

t = [Observations.get_product_list(obs) for obs in obs_table]
files = unique(vstack(t), keys='productFilename')

In [ ]:
mast_dir = 'mast:hst/product' # Download from MAST
data_dir = '/home/avinash/part4/HST/data/m82/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

manifest = Observations.download_products(
           files,
           productSubGroupDescription='RAW',
           filters=['F435W','F555W','F814W'],
           download_dir = data_dir,
           flat = True,
           )

In [ ]:
raw_files = glob("/home/avinash/part4/HST/data/m82/data/stage0/*raw.fits")

raw_f435w = []
raw_f555w = []
raw_f814w = []
for i in raw_files:
    if fits.open(i)[0].header['FILTER2']=='F435W':
        raw_f435w.append(i)
    if fits.open(i)[0].header['FILTER1']=='F555W':
        raw_f555w.append(i)
    elif fits.open(i)[0].header['FILTER2']=='F814W':
        raw_f814w.append(i)

In [ ]:
for file in raw_files:
     os.system(f'crds bestrefs --files {file} --sync-references=1 --update-bestrefs')

In [ ]:
for file in raw_f555w:
    calacs.calacs(file,single_core=True)

In [ ]:
flc_files = glob("../data/m82/data/stage0/*flc.fits")
for file in flc_files:
    os.replace(file,file.replace('stage0','stage2'))

In [ ]:
flc_files = glob("../data/m82/data/stage2/*flc.fits")

flc_f435w = []
flc_f555w = []
flc_f814w = []
for i in flc_files:
    if fits.open(i)[0].header['FILTER2']=='F435W':
        flc_f435w.append(i)
    if fits.open(i)[0].header['FILTER1']=='F555W':
        flc_f555w.append(i)
    elif fits.open(i)[0].header['FILTER2']=='F814W':
        flc_f814w.append(i)

In [ ]:
updatewcs.updatewcs('../data/m82/astrometry/h_m82_h_s05_drz_sci.fits')

In [ ]:
astrodrizzle.AstroDrizzle(input=flc_f814w, 
                          output= '../data/m82/astrometry/f814w_',mdriztab=False,
                          preserve=False, clean=True, runfile=None)

In [ ]:
from pydol.photometry.acs import acs_phot

In [ ]:
drz_path = '../data/m82/photometry/f814w__drc_sci'
out_dir = '../data/m82/photometry/'

In [ ]:
fits.open('../data/m82/photometry//j9l034fhq_flc/data.chip1.fits')

In [ ]:
hdul = fits.open('../data/m82/photometry/f814w__drc_sci.fits')
hdu = fits.PrimaryHDU(data=hdul[0].data, header=hdul[0].header)
hdul_n = fits.HDUList([hdu]) 
hdul_n.writeto('../data/m82/photometry/f814w__drc_sci.fits', overwrite=True)

In [ ]:
acs_phot(flc_f555w + flc_f814w ,
         filter='f555w_f814w',
         output_dir=out_dir,
         drz_path=drz_path,
         sharp_cut=0.2,
         crowd_cut=2.25,
         cat_name='')   

In [ ]:
"a"

'a'